# Solve Selection Bias using ADAPT library

In [ ]:
# set working directory
from random import SystemRandom
import pandas as pd
import numpy as np
import xgboost as xgb
# from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import RobustScaler

import os
import pickle
from sklearn.model_selection import train_test_split
import wandb

import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# from torch._C import float32
import argparse
from asyncio.log import logger
import os, math
import logging
import torch
import numpy as np
import json

import torch.nn as nn
import torch

import pickle
import json
import numpy as np
import random
from torch.utils.data import Dataset, DataLoader, random_split, WeightedRandomSampler
import torch
import os
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import load_iris
from sklearn.datasets import load_breast_cancer

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import torch.nn as nn
import torch
import math
import pandas as pd
import random

# Importing matplotlib and seaborn
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

import sys
import warnings

import numpy as np
import pandas as pd

from sklearn.neural_network import MLPClassifier
from sklearn.metrics import roc_auc_score
from hyperimpute.plugins.imputers import Imputers

from IPython.display import HTML, display
# import tabulate

from adapt.feature_based import DANN, WDGRL
from adapt.instance_based import KLIEP, KMM
import keras
from keras.optimizers.legacy import Adam
from keras import Model, Sequential
from keras.layers import Dense, Input, Dropout, Conv2D, MaxPooling2D, Flatten, Reshape, GaussianNoise, BatchNormalization

import utils
from utils import *
if not sys.warnoptions:
    warnings.simplefilter("ignore")

%matplotlib inline

In [ ]:
MINI_BATCH = 64
EPOCHS = 1000
LOAD = None
SEED = 42
REPEAT = 10

if torch.cuda.is_available():
    device = torch.device("cuda")  # Set device to GPU
    print("CUDA is available! Using GPU.")
else:
    device = torch.device("cpu")  # Set device to CPU
    print("CUDA is not available. Using CPU.")

torch.manual_seed(SEED)
np.random.seed(SEED)
torch.cuda.manual_seed(SEED)
random.seed(SEED)

param_grid = {
            'lambda': [0.05, 0.10],
            'hidden':[[50], [100], [100, 100]],
            'task':[[50, 1], [100, 1]],
            'lr': [0.0001, 0.0005]
        }
# params = {
#             'lambda': 0.05,
#             'hidden':[50],
#             'task':[50, 50, 1],
#             'lr': 0.0001
#         }
############################################
# initilising wandb
# wandb.init(project='SeletionBML', entity="jmdvinodjmd")
wandb.init(mode="disabled")
wandb.run.name = 'SB'
makedirs('./results/')
experimentID = LOAD
if experimentID is None:
    experimentID = int(SystemRandom().random()*100000)
# checkpoint
ckpt_path = os.path.join('./results/checkpoints/KMM_model.ckpt')
makedirs('./results/checkpoints/')
# set logger
log_path = os.path.join("./results/logs/" + "exp_KMM_" + str(experimentID) + ".log")
makedirs("./results/logs/")
logger = get_logger(logpath=log_path, filepath="exp_KMM_" + str(experimentID) + ".log", displaying=False)
logger.info("Experiment " + str(experimentID))
############################################

In [ ]:
def experiment(model_name, data, params, repeat=1):
    [X_train, y_train, s_train, X_val, y_val, s_val, X_test, y_test, s_test] = data

    # repeating experiment for a given number of times
    results_risk = {}
    for i in range(repeat):
        logger.info('Repeating: ' + str(i+1))
        results_risk[i] = {}
        #############################
        # train
        callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, min_delta=0.001)
        if model_name == 'KMM':
            model = KMM(estimator=get_mlp(params['task'], params['lr']), Xt=X_test, random_state=i, callbacks=callback)
        elif model_name == 'KLIEP':
            model = KLIEP(estimator=get_mlp(params['task'], params['lr']), Xt=X_test, random_state=i, callbacks=callback)
        model.fit(X_train[s_train==0], y_train[s_train==0], validation_data=(X_val[s_val==0], y_val[s_val==0]), epochs=EPOCHS, verbose=1)
        # evaluate model
        predictions = model.predict(X_val[s_val==0])
        results, _ = print_metrics_binary_classification(y_val[s_val==0], predictions, model_name, verbose=1, logger=logger, wandb=wandb)
        logger.info(model_name +':Val AUROC:' + str(results[model_name+' AUROC']))
        results_risk[i][model_name +'Val AUROC'] = results[model_name+' AUROC']

        # evaluate on test all
        predictions = model.predict(X_test)
        results, _ = print_metrics_binary_classification(y_test, predictions, model_name, verbose=1, logger=logger, wandb=wandb)
        logger.info(model_name +':Test AUROC:' + str(results[model_name+' AUROC']))
        results_risk[i][model_name +' Test AUROC'] = results[model_name+' AUROC']

        # evaluate on included
        predictions = model.predict(X_test[s_test==0])
        results, _ = print_metrics_binary_classification(y_test[s_test==0], predictions, model_name, verbose=1, logger=logger, wandb=wandb)
        logger.info(model_name +':Test AUROC(incl):' + str(results[model_name+' AUROC']))
        results_risk[i][model_name +' Test AUROC(incl)'] = results[model_name+' AUROC']
        # evaluate on excluded
        if (y_test[s_test==1].sum() == y_test[s_test==1].shape[0]) or (y_test[s_test==1].sum() == 0):
            # if there is only one class then AUROC is undefined so adding one sample from other class
            if (y_test[s_test==1].sum() == 0):
                predictions = model.predict(np.vstack((X_test[s_test==1], X_test[s_test==0][y_test[s_test==0]==1][:1,:].squeeze())))
                results, _ = print_metrics_binary_classification(np.append(y_test[s_test==1], 1), predictions, model_name, verbose=1, logger=logger, wandb=wandb)
                logger.info(model_name +':Test AUROC(excl):' + str(results[model_name+' AUROC']))
                results_risk[i][model_name +' Test AUROC(excl)'] = results[model_name+' AUROC']
            else:
                predictions = model.predict(np.vstack((X_test[s_test==1], X_test[s_test==0][y_test[s_test==0]==0][:1,:].squeeze())))
                results, _ = print_metrics_binary_classification(np.append(y_test[s_test==1], 0), predictions, model_name, verbose=1, logger=logger, wandb=wandb)
                logger.info(model_name +':Test AUROC(excl):' + str(results[model_name+' AUROC']))
                results_risk[i][model_name +' Test AUROC(excl)'] = results[model_name+' AUROC']
        else:
            predictions = model.predict(X_test[s_test==1])
            results, _ = print_metrics_binary_classification(y_test[s_test==1], predictions, model_name, verbose=1, logger=logger, wandb=wandb)
            logger.info(model_name +':Test AUROC(excl):' + str(results[model_name+' AUROC']))
            results_risk[i][model_name +' Test AUROC(excl)'] = results[model_name+' AUROC']

        ############################

    return results_risk

def study_effect(data_name, da_method, file_name, results_file, r, c, n, search_param=False):
    ''' 
    This function is used to study effect of (riks rate, dataset size etc.).
    It expects a set of datasets with some variations.
    '''
    logger.info('\n\n-------------N:'+str(n)+'--Risk Rate:' + str(r)+'--Censoring Rate:' + str(c)+'-------------------------.')

    results_sizes = {}
    for ni in n:
        for ci in c:
            for ri in r:
                # load data dictionary
                data_dict = get_data_dict(file_name, [ri], [ci], [ni])

                logger.info('-----Running for Size:'+str(ni)+'--Risk Rate:' + str(ri)+'--Censoring Rate:' + str(ci)+'\n-----------')
                [X_train, y_train, s_train, X_val, y_val, s_val, X_test, y_test, s_test] = data_dict[str(ni)+'R'+str(ri)+'C'+str(ci)]
                
                data = [X_train, y_train, s_train, X_val, y_val, s_val, X_test, y_test, s_test]
                
                #############################
                # Reading hyperparameters from the JSON file
                with open('best_hyperparams.json', 'r') as json_file:
                    best_hyperparams = json.load(json_file)
                if (da_method+data_name+str(ni)+'R'+str(ri)+'C'+str(ci) not in best_hyperparams) or search_param:
                    # hyperparameter tuning 
                    logger.info('Finding best hyperparams.')
                    params, best_score, results = grid_search_kmm_kliep(da_method, X_train[s_train==0], y_train[s_train==0], X_val[s_val==0], y_val[s_val==0], X_test, ckpt_path, param_grid, EPOCHS, logger, wandb, device)
                    logger.info('Hyperparam tuning:')
                    logger.info(results)

                    best_hyperparams[da_method+data_name+str(ni)+'R'+str(ri)+'C'+str(ci)] = {'params': params}
                    # save best params
                    with open('best_hyperparams.json', 'w') as json_file:
                        json.dump(best_hyperparams, json_file)
                else:
                    logger.info('Accessing the existing best hyperparams.')
                    params = best_hyperparams[da_method+data_name+str(ni)+'R'+str(ri)+'C'+str(ci)]['params']

                ################################
                # run experiments and repeat for given number of times
                results = experiment(da_method, data, params, repeat=REPEAT)
                logger.info('\n\nBest params for risk:\n' + str(params))
                logger.info(results)
                results_sizes[str(ni)+'R'+str(ri)+'C'+str(ci)] = results

                # save results
                dict_to_file(results_file, results_sizes)
                ################################

    logger.info('\n\n------------------- Experiments ended-------------------.\n'+str(results_sizes)+'\n------------------------------------------------\n\n')

    return results_sizes


## Synthetic

In [ ]:
results_sizes = study_effect('synthetic', 'KMM', 'selection_bias_data.pkl', 'results_KMM', [.05, .1, .2, .3, .4], c=[.05, .1, .2, .3, .4], n=[1000, 2000, 3000, 4000, 5000], search_param=False)

In [ ]:
results_sizes = study_effect('synthetic', 'KLIEP', 'selection_bias_data.pkl', 'results_KLIEP-T', [.05, .1, .2, .3, .4], c=[.05, .1, .2, .3, .4], n=[1000, 2000, 3000, 4000, 5000], search_param=True)

## Diabetes

In [ ]:
results_sizes = study_effect('diabetes', 'KMM', 'diabetes_bias_data.pkl', 'results_KMM-diabetes', r=[.05, .1, .2, .3, .4], c=[.05, .1, .2, .3, .4], n=[25000, 10000, 5000, 2000, 1000], search_param=False)

In [ ]:
results_sizes = study_effect('diabetes', 'KLIEP', 'diabetes_bias_data.pkl', 'results_KLIEP-diabetes-T', r=[.05, .1, .2, .3, .4], c=[.05, .1, .2, .3, .4], n=[25000, 10000, 5000, 2000, 1000], search_param=True)

## Covid

In [ ]:
results_sizes = study_effect('covid', 'KMM', 'covid_bias_data.pkl', 'results_KMM-covid', r=[.05, .1, .2, .3, .4], c=[.05, .1, .2, .3, .4], n=[15000, 10000, 5000, 2000, 1000], search_param=False)

In [ ]:
results_sizes = study_effect('covid', 'KLIEP', 'covid_bias_data.pkl', 'results_KLIEP-covid-T', r=[.05, .1, .2, .3, .4], c=[.05, .1, .2, .3, .4], n=[15000, 10000, 5000, 2000, 1000], search_param=True)